In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1683127,2021-09-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1683128,2021-09-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1683129,2021-09-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1683130,2021-09-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
43963,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
43965,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
43967,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
43969,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
43971,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1026131,2021-09-23,Arkansas,Arkansas,5001,3358,56.00,5000,Arkansas,AR,Arkansas,State,3017804
1026133,2021-09-24,Arkansas,Arkansas,5001,3363,56.00,5000,Arkansas,AR,Arkansas,State,3017804
1026135,2021-09-25,Arkansas,Arkansas,5001,3369,56.00,5000,Arkansas,AR,Arkansas,State,3017804
1026137,2021-09-26,Arkansas,Arkansas,5001,3369,56.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1683127,2021-09-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1683128,2021-09-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1683129,2021-09-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1683130,2021-09-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-09-27') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
615,2021-09-27,Snohomish,Washington,58286,690.00,53061,WA,County,822083,7090.04,83.93
1228,2021-09-27,Cook,Illinois,617360,11563.00,17031,IL,County,5150233,11987.03,224.51
1840,2021-09-27,Orange,California,317998,5409.00,6059,CA,County,3175692,10013.50,170.33
2451,2021-09-27,Maricopa,Arizona,689931,11371.00,4013,AZ,County,4485414,15381.66,253.51
3062,2021-09-27,Los Angeles,California,1455864,26035.00,6037,CA,County,10039107,14501.93,259.34
...,...,...,...,...,...,...,...,...,...,...,...
1681855,2021-09-27,Wheeler,Oregon,65,1.00,41069,OR,County,1332,4879.88,75.08
1682205,2021-09-27,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
1682524,2021-09-27,Esmeralda,Nevada,48,0.00,32009,NV,County,873,5498.28,0.00
1682839,2021-09-27,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
615,2021-09-27,Snohomish,Washington,58286,690.00,53061,WA,County,822083,7090.04,83.93,83.93,7090.04
1228,2021-09-27,Cook,Illinois,617360,11563.00,17031,IL,County,5150233,11987.03,224.51,224.51,11987.03
1840,2021-09-27,Orange,California,317998,5409.00,6059,CA,County,3175692,10013.50,170.33,170.33,10013.50
2451,2021-09-27,Maricopa,Arizona,689931,11371.00,4013,AZ,County,4485414,15381.66,253.51,253.51,15381.66
3062,2021-09-27,Los Angeles,California,1455864,26035.00,6037,CA,County,10039107,14501.93,259.34,259.34,14501.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681855,2021-09-27,Wheeler,Oregon,65,1.00,41069,OR,County,1332,4879.88,75.08,75.08,4879.88
1682205,2021-09-27,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
1682524,2021-09-27,Esmeralda,Nevada,48,0.00,32009,NV,County,873,5498.28,0.00,0.00,5498.28
1682839,2021-09-27,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1073264,2021-09-27,Hale,Alabama,2930,85.00,1065,AL,County,14651,19998.63,580.17,580.17,19998.63,1
1168909,2021-09-27,Clarke,Alabama,4640,79.00,1025,AL,County,23622,19642.71,334.43,334.43,19642.71,2
743091,2021-09-27,Etowah,Alabama,19341,462.00,1055,AL,County,102268,18912.07,451.75,451.75,18912.07,3
303316,2021-09-27,Calhoun,Alabama,21297,410.00,1015,AL,County,113605,18746.53,360.90,360.90,18746.53,4
623016,2021-09-27,Franklin,Alabama,5805,101.00,1059,AL,County,31362,18509.66,322.05,322.05,18509.66,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599253,2021-09-27,Weston,Wyoming,891,8.00,56045,WY,County,6927,12862.71,115.49,115.49,12862.71,19
903048,2021-09-27,Johnson,Wyoming,1061,13.00,56019,WY,County,8445,12563.65,153.94,153.94,12563.65,20
1274991,2021-09-27,Lincoln,Wyoming,2321,15.00,56023,WY,County,19830,11704.49,75.64,75.64,11704.49,21
1024495,2021-09-27,Sublette,Wyoming,1148,12.00,56035,WY,County,9831,11677.35,122.06,122.06,11677.35,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
838581,2021-09-27,Lowndes,Alabama,1708,58.00,1085,AL,County,9726,17561.18,596.34,596.34,17561.18,13,1
1073264,2021-09-27,Hale,Alabama,2930,85.00,1065,AL,County,14651,19998.63,580.17,580.17,19998.63,1,2
904698,2021-09-27,Greene,Alabama,1214,43.00,1063,AL,County,8111,14967.33,530.14,530.14,14967.33,49,3
355300,2021-09-27,Walker,Alabama,10604,330.00,1127,AL,County,63521,16693.69,519.51,519.51,16693.69,30,4
837479,2021-09-27,Crenshaw,Alabama,2387,70.00,1041,AL,County,13772,17332.27,508.28,508.28,17332.27,18,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599253,2021-09-27,Weston,Wyoming,891,8.00,56045,WY,County,6927,12862.71,115.49,115.49,12862.71,19,19
1242954,2021-09-27,Uinta,Wyoming,3473,23.00,56041,WY,County,20226,17170.97,113.72,113.72,17170.97,4,20
1274991,2021-09-27,Lincoln,Wyoming,2321,15.00,56023,WY,County,19830,11704.49,75.64,75.64,11704.49,21,21
835275,2021-09-27,Albany,Wyoming,5719,26.00,56001,WY,County,38880,14709.36,66.87,66.87,14709.36,11,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,83.93,7090.04,26,29
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,83.93,7090.04,26,29
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,83.93,7090.04,26,29
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,83.93,7090.04,26,29
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,83.93,7090.04,26,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679221,2021-09-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1679222,2021-09-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1679223,2021-09-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1679224,2021-09-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
1068812,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,580.17,19998.63,2,1,1.00
1068813,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,580.17,19998.63,2,1,0.00
1068814,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,580.17,19998.63,2,1,0.00
1068815,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,580.17,19998.63,2,1,0.00
1068816,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,580.17,19998.63,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296555,2021-09-23,Crook,Wyoming,733,16.00,56011,WY,County,7584,9665.08,210.97,210.97,9955.17,7,23,10.00
1296556,2021-09-24,Crook,Wyoming,744,16.00,56011,WY,County,7584,9810.13,210.97,210.97,9955.17,7,23,11.00
1296557,2021-09-25,Crook,Wyoming,744,16.00,56011,WY,County,7584,9810.13,210.97,210.97,9955.17,7,23,0.00
1296558,2021-09-26,Crook,Wyoming,744,16.00,56011,WY,County,7584,9810.13,210.97,210.97,9955.17,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
835223,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,596.34,17561.18,1,13,1.00,0.00
835224,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,596.34,17561.18,1,13,0.00,0.00
835225,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,596.34,17561.18,1,13,0.00,0.00
835226,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,596.34,17561.18,1,13,0.00,0.00
835227,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,596.34,17561.18,1,13,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351366,2021-09-23,Teton,Wyoming,4870,12.00,56039,WY,County,23464,20755.20,51.14,51.14,20878.79,23,1,19.00,0.00
351367,2021-09-24,Teton,Wyoming,4883,12.00,56039,WY,County,23464,20810.60,51.14,51.14,20878.79,23,1,13.00,0.00
351368,2021-09-25,Teton,Wyoming,4883,12.00,56039,WY,County,23464,20810.60,51.14,51.14,20878.79,23,1,0.00,0.00
351369,2021-09-26,Teton,Wyoming,4883,12.00,56039,WY,County,23464,20810.60,51.14,51.14,20878.79,23,1,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-09-27') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
424460,2021-09-27,Imperial,California,34644,755.00,6025,CA,County,181215,19117.62,416.63,416.63,19117.62,1,3,182.00,1.00
3062,2021-09-27,Los Angeles,California,1455864,26035.00,6037,CA,County,10039107,14501.93,259.34,259.34,14501.93,2,10,983.00,22.00
210762,2021-09-27,San Bernardino,California,350405,5545.00,6071,CA,County,2180085,16073.00,254.35,254.35,16073.00,3,4,1358.00,5.00
93961,2021-09-27,Stanislaus,California,82039,1262.00,6099,CA,County,550660,14898.30,229.18,229.18,14898.30,4,7,562.00,6.00
745251,2021-09-27,Inyo,California,1713,39.00,6027,CA,County,18039,9496.09,216.20,216.20,9496.09,5,36,28.00,0.00
74685,2021-09-27,San Joaquin,California,99013,1642.00,6077,CA,County,762148,12991.31,215.44,215.44,12991.31,6,17,662.00,1.00
43392,2021-09-27,Riverside,California,361852,4963.00,6065,CA,County,2470546,14646.64,200.89,200.89,14646.64,7,8,1794.00,12.00
675037,2021-09-27,Merced,California,40461,554.00,6047,CA,County,277680,14571.09,199.51,199.51,14571.09,8,9,341.00,2.00
42252,2021-09-27,Fresno,California,138323,1939.00,6019,CA,County,999101,13844.75,194.07,194.07,13844.75,9,13,936.00,12.00
116598,2021-09-27,Tulare,California,74793,904.00,6107,CA,County,466195,16043.29,193.91,193.91,16043.29,10,5,651.00,6.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1587985,2021-09-27,Lassen,California,7089,36.00,6035,CA,County,30573,23187.13,117.75,117.75,23187.13,28,1,0.00,0.00
904640,2021-09-27,Kings,California,30713,293.00,6031,CA,County,152940,20081.73,191.58,191.58,20081.73,11,2,294.00,0.00
424460,2021-09-27,Imperial,California,34644,755.00,6025,CA,County,181215,19117.62,416.63,416.63,19117.62,1,3,182.00,1.00
210762,2021-09-27,San Bernardino,California,350405,5545.00,6071,CA,County,2180085,16073.00,254.35,254.35,16073.00,3,4,1358.00,5.00
116598,2021-09-27,Tulare,California,74793,904.00,6107,CA,County,466195,16043.29,193.91,193.91,16043.29,10,5,651.00,6.00
265221,2021-09-27,Kern,California,137884,1540.00,6029,CA,County,900202,15317.01,171.07,171.07,15317.01,13,6,952.00,8.00
93961,2021-09-27,Stanislaus,California,82039,1262.00,6099,CA,County,550660,14898.30,229.18,229.18,14898.30,4,7,562.00,6.00
43392,2021-09-27,Riverside,California,361852,4963.00,6065,CA,County,2470546,14646.64,200.89,200.89,14646.64,7,8,1794.00,12.00
675037,2021-09-27,Merced,California,40461,554.00,6047,CA,County,277680,14571.09,199.51,199.51,14571.09,8,9,341.00,2.00
3062,2021-09-27,Los Angeles,California,1455864,26035.00,6037,CA,County,10039107,14501.93,259.34,259.34,14501.93,2,10,983.00,22.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
424460,2021-09-27,Imperial,California,34644,755.00,6025,CA,County,181215,19117.62,416.63,416.63,19117.62,1,3,182.00,1.00
3062,2021-09-27,Los Angeles,California,1455864,26035.00,6037,CA,County,10039107,14501.93,259.34,259.34,14501.93,2,10,983.00,22.00
210762,2021-09-27,San Bernardino,California,350405,5545.00,6071,CA,County,2180085,16073.00,254.35,254.35,16073.00,3,4,1358.00,5.00
93961,2021-09-27,Stanislaus,California,82039,1262.00,6099,CA,County,550660,14898.30,229.18,229.18,14898.30,4,7,562.00,6.00
745251,2021-09-27,Inyo,California,1713,39.00,6027,CA,County,18039,9496.09,216.20,216.20,9496.09,5,36,28.00,0.00
74685,2021-09-27,San Joaquin,California,99013,1642.00,6077,CA,County,762148,12991.31,215.44,215.44,12991.31,6,17,662.00,1.00
43392,2021-09-27,Riverside,California,361852,4963.00,6065,CA,County,2470546,14646.64,200.89,200.89,14646.64,7,8,1794.00,12.00
675037,2021-09-27,Merced,California,40461,554.00,6047,CA,County,277680,14571.09,199.51,199.51,14571.09,8,9,341.00,2.00
42252,2021-09-27,Fresno,California,138323,1939.00,6019,CA,County,999101,13844.75,194.07,194.07,13844.75,9,13,936.00,12.00
116598,2021-09-27,Tulare,California,74793,904.00,6107,CA,County,466195,16043.29,193.91,193.91,16043.29,10,5,651.00,6.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5582,09/23/21,Lassen,7053,35.00,23069.37,114.48,28,1,18.00,1.00
5583,09/24/21,Lassen,7089,36.00,23187.13,117.75,28,1,36.00,1.00
5584,09/25/21,Lassen,7089,36.00,23187.13,117.75,28,1,0.00,0.00
5585,09/26/21,Lassen,7089,36.00,23187.13,117.75,28,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,3,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,3,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,3,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
5668,09/23/21,Tulare,74009,896.00,15875.12,192.19,10,5,593.00,0.00
5669,09/24/21,Tulare,74142,898.00,15903.65,192.62,10,5,133.00,2.00
5670,09/25/21,Tulare,74142,898.00,15903.65,192.62,10,5,0.00,0.00
5671,09/26/21,Tulare,74142,898.00,15903.65,192.62,10,5,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)